# project

## project的三个需求
1. 表示数字艺术品
2. 存储数字艺术品
3. 交换数字艺术品

## 安全需求分析
1. 需要记录用户和用户拥有的艺术品
    - 需要将用户和用户的独特签名对应起来
    - 需要在艺术品的存储中添加用户的独特签名
    - 将签名添加到艺术品的算法需要加密，防止攻击者修改艺术品数据内的独特签名
    
2. 数据交换
    - 交换艺术品时，需要将交换的请求加密，防止被中途修改
    - ACID原则
        - 原子性（A）：一个事务的所有系列操作步骤被看成一个动作，所有的步骤要么全部完成，要么一个也不会完成。如果在事务过程中发生错误，则会回滚到事务开始前的状态，将要被改变的数据库记录不会被改变。
        - 一致性（C）：一致性是指在事务开始之前和事务结束以后，数据库的完整性约束没有被破坏，即数据库事务不能破坏关系数据的完整性及业务逻辑上的一致性。
        - 隔离性（I）：主要用于实现并发控制，隔离能够确保并发执行的事务按顺序一个接一个地执行。通过隔离，一个未完成事务不会影响另外一个未完成事务。
        - 持久性（D）：一旦一个事务被提交，它应该持久保存，不会因为与其他操作冲突而取消这个事务。

3. 数据存储的安全性
    - NFT的不可篡改性依靠区块链的安全特性来实现。这里，我们将区块链替换为了一般的数据库，这将使平台失去去中心化的优势。为了仍然保障安全性，需要做出以下假设：
        - 部署数据库的设备不会被攻破
        - 数据库管理员的密码不会被泄露
        - 数据库管理员不会恶意修改数据库

4. 需要保护所有用户的独特签名

## 重要reference
- [中泰证券NFT技术分析](https://dfscdn.dfcfw.com/download/A2_cms_f_20220216123508144922&direct=1&abc3847.pdf)
- [我的总结（祥见参考列表） - csdn](https://blog.csdn.net/weixin_39591031/article/details/124138855)


In [21]:
import time
import rsa

In [22]:
(pubkey, privkey) = rsa.newkeys(1024)

In [1]:
pub = pubkey.save_pkcs1()
pubkey, pub

NameError: name 'pubkey' is not defined

In [15]:
class User:
    def __init__(self):
        # generate unique user signature
        self.signature = self.gen_signature()
        self.keys = self.gen_keys()
        self.image_list = self.get_image_list()
    
    @staticmethod
    def gen_signature(self):
        return round(time.time() * 10000000)

    def gen_keys(self):
        keys = {}
        keys['RSA_priv'] = 0
        keys['RSA_pub'] = 0
        keys['ARS_priv'] = 0
        keys['ARS_pub'] = 0

    def get_image_list(self):
        # retrieve user's images from database
        pass

In [17]:
xav = User()

In [18]:
xav.signature


16497800606401128